In [1]:
import pandas as pd, numpy as np

# Add 'industry oil' to coverage input data

In [2]:
## pull in (previously final) input data sheet
outputs_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/'
input_data_path = outputs_dir + 'lsff_input_coverage_data.csv'

input_data = pd.read_csv(input_data_path)

loc_id_map = {i:j for(i,j) in zip(input_data.location_name, input_data.location_id) }

In [3]:
# pull in (previously final) version of input data sheet with metadata
w_meta = pd.read_csv(outputs_dir + 'lsff_input_coverage_data_with_metadata.csv')

In [4]:
# pull in new numbers from Jonathan / industry
inputs_dir = "/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/"
new_data = pd.read_excel(inputs_dir + "20210411_LSFF_input_data_for_IHME.xlsx", header = [1,2,3])

In [5]:
# subset to the oil numbers we're interested in
new_data = new_data.set_index(new_data.columns[0])

new_oil = new_data['Baseline coverage']['Oil\n(percent of population eating specified oil)'][['Fortifiable oil.1','Compliance % of fortifiable']]

In [7]:
# pull columns we're interested in
rename_oil = {
    ('Location', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2'): 'location_name',
 'Fortifiable oil.1': 'percent of population eating industrially produced vehicle',
 'Compliance % of fortifiable': 'percent of population eating fortified vehicle'
}

new_oil = new_oil.reset_index().rename(columns=rename_oil)

In [9]:
# reshape to our format
new_oil = pd.melt(new_oil, id_vars = 'location_name',
        value_vars = ['percent of population eating industrially produced vehicle',
       'percent of population eating fortified vehicle'],
       var_name='value_description',value_name='value_mean')

In [10]:
# fix c'ote divoire 
new_oil.loc[new_oil.location_name=="CÃ´te d'Ivoire","location_name"] = "Côte d'Ivoire"

In [11]:
assert(set(new_oil.location_name).symmetric_difference(set(input_data.location_name))==set())

In [12]:
new_oil['vehicle'] = 'industry oil'
new_oil['nutrient'] = 'vitamin a'
new_oil['u5_applicable'] = True
new_oil['wra_applicable'] = True
new_oil['sub_population'] = np.NaN
new_oil['location_id'] = new_oil.location_name.map(loc_id_map)

In [13]:
new_oil.head()

,location_name,value_description,value_mean,vehicle,nutrient,u5_applicable,wra_applicable,sub_population,location_id
0,Angola,percent of population eating industrially prod...,72.320,industry oil,vitamin a,True,True,NaN,168
1,Bangladesh,percent of population eating industrially prod...,87.516,industry oil,vitamin a,True,True,NaN,161
2,Burkina Faso,percent of population eating industrially prod...,73.760,industry oil,vitamin a,True,True,NaN,201
3,Côte d'Ivoire,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,205
4,Cameroon,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,202


In [14]:
#from our data, select extracted rows of data (no regression or multiplicative results)
#then drop any rows duplicated for different nutrients
w_meta = w_meta[w_meta.estimation_status=='na'].drop(columns = 'nutrient').drop_duplicates()

In [15]:
w_meta['scale_over_mean'] = (w_meta.value_975_percentile - w_meta.value_025_percentile) / w_meta.value_mean
r = w_meta.loc[(w_meta.scale_over_mean!=np.inf) & (w_meta.scale_over_mean!=0),['vehicle','scale_over_mean']]

In [16]:
r_mean = r.scale_over_mean.mean()
r.loc[r.vehicle=="maize flour",'scale_over_mean'] = r_mean

In [17]:
r = r.groupby('vehicle').mean().dropna().rename(columns={'scale_over_mean':'r'}).reset_index()

In [18]:
r

,vehicle,r
0,maize flour,0.564982
1,oil,0.379944
2,wheat flour,0.688639


In [26]:
new_oil['r'] = 0.379944

In [27]:
new_oil

,location_name,value_description,value_mean,vehicle,nutrient,u5_applicable,wra_applicable,sub_population,location_id,r
0,Angola,percent of population eating industrially prod...,72.3200,industry oil,vitamin a,True,True,NaN,168,0.379944
1,Bangladesh,percent of population eating industrially prod...,87.5160,industry oil,vitamin a,True,True,NaN,161,0.379944
2,Burkina Faso,percent of population eating industrially prod...,73.7600,industry oil,vitamin a,True,True,NaN,201,0.379944
3,Côte d'Ivoire,percent of population eating industrially prod...,72.0000,industry oil,vitamin a,True,True,NaN,205,0.379944
4,Cameroon,percent of population eating industrially prod...,72.0000,industry oil,vitamin a,True,True,NaN,202,0.379944
5,China,percent of population eating industrially prod...,96.5300,industry oil,vitamin a,True,True,NaN,6,0.379944
6,Democratic Republic of the Congo,percent of population eating industrially prod...,76.7200,industry oil,vitamin a,True,True,NaN,171,0.379944
7,Egypt,percent of population eating industrially prod...,64.0000,industry oil,vitamin a,True,True,NaN,141,0.379944
8,Ethiopia,percent of population eating industrially prod...,71.2500,industry oil,vitamin a,True,True,NaN,179,0.379944
9,Ghana,percent of population eating industrially prod...,78.3920,industry oil,vitamin a,True,True,NaN,207,0.379944


In [28]:
new_oil['value_025_percentile'] = np.clip(new_oil.value_mean - (new_oil.r * new_oil.value_mean)/2, 0, 100)
new_oil['value_975_percentile'] = np.clip(new_oil.value_mean + (new_oil.r * new_oil.value_mean)/2, 0, 100)

In [29]:
new_oil

,location_name,value_description,value_mean,vehicle,nutrient,u5_applicable,wra_applicable,sub_population,location_id,r,value_025_percentile,value_975_percentile
0,Angola,percent of population eating industrially prod...,72.3200,industry oil,vitamin a,True,True,NaN,168,0.379944,58.581225,86.058775
1,Bangladesh,percent of population eating industrially prod...,87.5160,industry oil,vitamin a,True,True,NaN,161,0.379944,70.890410,100.000000
2,Burkina Faso,percent of population eating industrially prod...,73.7600,industry oil,vitamin a,True,True,NaN,201,0.379944,59.747665,87.772335
3,Côte d'Ivoire,percent of population eating industrially prod...,72.0000,industry oil,vitamin a,True,True,NaN,205,0.379944,58.322016,85.677984
4,Cameroon,percent of population eating industrially prod...,72.0000,industry oil,vitamin a,True,True,NaN,202,0.379944,58.322016,85.677984
5,China,percent of population eating industrially prod...,96.5300,industry oil,vitamin a,True,True,NaN,6,0.379944,78.192003,100.000000
6,Democratic Republic of the Congo,percent of population eating industrially prod...,76.7200,industry oil,vitamin a,True,True,NaN,171,0.379944,62.145348,91.294652
7,Egypt,percent of population eating industrially prod...,64.0000,industry oil,vitamin a,True,True,NaN,141,0.379944,51.841792,76.158208
8,Ethiopia,percent of population eating industrially prod...,71.2500,industry oil,vitamin a,True,True,NaN,179,0.379944,57.714495,84.785505
9,Ghana,percent of population eating industrially prod...,78.3920,industry oil,vitamin a,True,True,NaN,207,0.379944,63.499715,93.284285


In [30]:
new_oil[input_data.columns]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,wra_applicable
0,168,Angola,NaN,industry oil,percent of population eating industrially prod...,vitamin a,72.3200,58.581225,86.058775,True,True
1,161,Bangladesh,NaN,industry oil,percent of population eating industrially prod...,vitamin a,87.5160,70.890410,100.000000,True,True
2,201,Burkina Faso,NaN,industry oil,percent of population eating industrially prod...,vitamin a,73.7600,59.747665,87.772335,True,True
3,205,Côte d'Ivoire,NaN,industry oil,percent of population eating industrially prod...,vitamin a,72.0000,58.322016,85.677984,True,True
4,202,Cameroon,NaN,industry oil,percent of population eating industrially prod...,vitamin a,72.0000,58.322016,85.677984,True,True
5,6,China,NaN,industry oil,percent of population eating industrially prod...,vitamin a,96.5300,78.192003,100.000000,True,True
6,171,Democratic Republic of the Congo,NaN,industry oil,percent of population eating industrially prod...,vitamin a,76.7200,62.145348,91.294652,True,True
7,141,Egypt,NaN,industry oil,percent of population eating industrially prod...,vitamin a,64.0000,51.841792,76.158208,True,True
8,179,Ethiopia,NaN,industry oil,percent of population eating industrially prod...,vitamin a,71.2500,57.714495,84.785505,True,True
9,207,Ghana,NaN,industry oil,percent of population eating industrially prod...,vitamin a,78.3920,63.499715,93.284285,True,True


In [33]:
output = input_data.append(new_oil)

In [34]:
output

,location_id,location_name,nutrient,r,sub_population,u5_applicable,value_025_percentile,value_975_percentile,value_description,value_mean,vehicle,wra_applicable
0,179,Ethiopia,na,NaN,NaN,True,24.950317,93.577380,percent of population eating vehicle,59.263848,maize flour,True
1,179,Ethiopia,na,NaN,NaN,True,11.437632,42.897396,percent of population eating industrially prod...,27.167514,maize flour,True
2,179,Ethiopia,folic acid,NaN,NaN,True,0.000000,0.000000,percent of population eating fortified vehicle,0.000000,maize flour,True
3,179,Ethiopia,iron,NaN,NaN,True,0.000000,0.000000,percent of population eating fortified vehicle,0.000000,maize flour,True
4,179,Ethiopia,vitamin a,NaN,NaN,True,0.000000,0.000000,percent of population eating fortified vehicle,0.000000,maize flour,True
...,...,...,...,...,...,...,...,...,...,...,...,...
45,196,South Africa,vitamin a,0.379944,NaN,True,2.430084,3.569916,percent of population eating fortified vehicle,3.000000,industry oil,True
46,522,Sudan,vitamin a,0.379944,NaN,True,0.000000,0.000000,percent of population eating fortified vehicle,0.000000,industry oil,True
47,190,Uganda,vitamin a,0.379944,NaN,True,44.065523,64.734477,percent of population eating fortified vehicle,54.400000,industry oil,True
48,189,United Republic of Tanzania,vitamin a,0.379944,NaN,True,43.417501,63.782499,percent of population eating fortified vehicle,53.600000,industry oil,True


In [35]:
output.columns

Index(['location_id', 'location_name', 'nutrient', 'r', 'sub_population',
       'u5_applicable', 'value_025_percentile', 'value_975_percentile',
       'value_description', 'value_mean', 'vehicle', 'wra_applicable'],
      dtype='object')

In [40]:
output.loc[(output.value_mean >= output.value_975_percentile) & (output.value_description!='percent of population eating vehicle')]

,location_id,location_name,nutrient,r,sub_population,u5_applicable,value_025_percentile,value_975_percentile,value_description,value_mean,vehicle,wra_applicable
2,179,Ethiopia,folic acid,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,maize flour,True
3,179,Ethiopia,iron,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,maize flour,True
4,179,Ethiopia,vitamin a,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,maize flour,True
5,179,Ethiopia,zinc,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,maize flour,True
9,179,Ethiopia,vitamin a,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,oil,True
13,179,Ethiopia,folic acid,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True
14,179,Ethiopia,iron,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True
15,179,Ethiopia,vitamin a,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True
16,179,Ethiopia,zinc,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True
56,165,Pakistan,vitamin a,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True


In [41]:
output.loc[(output.value_mean <= output.value_025_percentile) & (output.value_description!='percent of population eating vehicle')]

,location_id,location_name,nutrient,r,sub_population,u5_applicable,value_025_percentile,value_975_percentile,value_description,value_mean,vehicle,wra_applicable
2,179,Ethiopia,folic acid,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,maize flour,True
3,179,Ethiopia,iron,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,maize flour,True
4,179,Ethiopia,vitamin a,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,maize flour,True
5,179,Ethiopia,zinc,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,maize flour,True
9,179,Ethiopia,vitamin a,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,oil,True
13,179,Ethiopia,folic acid,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True
14,179,Ethiopia,iron,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True
15,179,Ethiopia,vitamin a,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True
16,179,Ethiopia,zinc,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True
56,165,Pakistan,vitamin a,NaN,NaN,True,0.0,0.0,percent of population eating fortified vehicle,0.0,wheat flour,True


In [44]:
output.to_csv('/ihme/homes/beatrixh/repos/scratch/lsff_input_coverage_data.csv')